In [1]:
# Paso 2: Crear Directorios de Trabajo
import os

# Define las rutas
pdfs_dir = 'documents'  # Carpeta que contiene los PDFs
output_text_dir = 'textos'  # Carpeta para guardar los textos extraídos
output_parquet_path = 'datos_procesados.parquet'  # Ruta para el archivo .parquet

# Crea las carpetas si no existen
os.makedirs(pdfs_dir, exist_ok=True)
os.makedirs(output_text_dir, exist_ok=True)

# Paso 3: Instalar Dependencias
#!pip install pdfminer.six spacy pyspellchecker transformers torch pandas pyarrow tqdm

# Paso 4: Descargar Modelos Necesarios
import spacy

# Descargar el modelo BERT en español (BETO)
from transformers import BertTokenizer, BertModel

# Usaremos el modelo BETO para mejor compatibilidad con español
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
modelo = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

# Paso 5: Definir Funciones de Procesamiento
import re
import json
import unicodedata
from pathlib import Path
from typing import List, Dict

import pandas as pd
from tqdm import tqdm
from pdfminer.high_level import extract_text
import spacy
from spellchecker import SpellChecker
from transformers import BertTokenizer, BertModel
import torch

# Verifica si CUDA está disponible y configura el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

# Cargar el modelo de spaCy
nlp = spacy.load('es_core_news_sm')

# Inicializar el corrector ortográfico para español
spell = SpellChecker(language='es')

# Cargar el tokenizador y el modelo BERT en español, y moverlo al dispositivo adecuado
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
modelo = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased').to(device)

def extraer_texto_pdf(ruta_pdf: str) -> str:
    """
    Extrae texto de un archivo PDF.

    Args:
        ruta_pdf (str): Ruta al archivo PDF.

    Returns:
        str: Texto extraído del PDF.
    """
    try:
        texto = extract_text(ruta_pdf)
        return texto
    except Exception as e:
        print(f'Error extrayendo texto de {ruta_pdf}: {e}')
        return ''

def limpiar_y_normalizar_texto(texto: str) -> str:
    """
    Limpia y normaliza el texto extraído.

    Args:
        texto (str): Texto bruto extraído del PDF.

    Returns:
        str: Texto limpio y normalizado.
    """
    texto = re.sub(r'\s+', ' ', texto)  # Reemplaza múltiples espacios por uno solo
    texto = texto.strip()
    texto = unicodedata.normalize('NFC', texto)  # Normaliza caracteres Unicode
    return texto

def tokenizar_y_lemmatizar(texto: str, nlp) -> List[str]:
    """
    Tokeniza y lematiza el texto utilizando spaCy.

    Args:
        texto (str): Texto limpio y normalizado.
        nlp: Objeto de spaCy para procesamiento.

    Returns:
        list: Lista de tokens lematizados.
    """
    doc = nlp(texto)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

def corregir_ortografia(palabras: List[str], spell: SpellChecker) -> List[str]:
    """
    Corrige la ortografía de una lista de palabras.

    Args:
        palabras (list): Lista de palabras lematizadas.
        spell (SpellChecker): Objeto SpellChecker.

    Returns:
        list: Lista de palabras corregidas.
    """
    palabras_corregidas = []
    for palabra in palabras:
        correccion = spell.correction(palabra)
        if correccion:
            if palabra.isupper():
                correccion = correccion.upper()
            elif palabra[0].isupper():
                correccion = correccion.capitalize()
            palabras_corregidas.append(correccion)
        else:
            palabras_corregidas.append(palabra)
    return palabras_corregidas

def generar_embeddings(texto: str, tokenizer, modelo, device) -> List[float]:
    """
    Genera embeddings para el texto dado utilizando BERT en GPU si está disponible.

    Args:
        texto (str): Texto preprocesado.
        tokenizer: Tokenizador de BERT.
        modelo: Modelo de BERT.
        device: Dispositivo a utilizar ('cuda' o 'cpu').

    Returns:
        list: Embeddings generados por BERT como lista de floats.
    """
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        salidas = modelo(**inputs)
    embeddings = salidas.last_hidden_state.mean(dim=1)  # Promedio de las representaciones de tokens
    return embeddings.squeeze().cpu().numpy().tolist()

def procesar_pdf(ruta_pdf: str, nlp, spell, tokenizer, modelo, device) -> Dict:
    """
    Procesa un documento PDF completo.

    Args:
        ruta_pdf (str): Ruta al archivo PDF.
        nlp: Objeto de spaCy para procesamiento.
        spell (SpellChecker): Objeto SpellChecker.
        tokenizer: Tokenizador de BERT.
        modelo: Modelo de BERT.
        device: Dispositivo a utilizar ('cuda' o 'cpu').

    Returns:
        dict: Información procesada del documento.
    """
    nombre_archivo = os.path.basename(ruta_pdf)
    texto = extraer_texto_pdf(ruta_pdf)
    texto = limpiar_y_normalizar_texto(texto)
    tokens = tokenizar_y_lemmatizar(texto, nlp)
    palabras_corregidas = corregir_ortografia(tokens, spell)
    texto_preprocesado = ' '.join(palabras_corregidas)
    embeddings = generar_embeddings(texto_preprocesado, tokenizer, modelo, device)
    
    return {
        "nombre_archivo": nombre_archivo,
        "ruta": ruta_pdf,
        "embeddings": embeddings,
        "texto": texto_preprocesado
    }

def guardar_parquet(datos: List[Dict], ruta_parquet: str):
    """
    Guarda los datos procesados en un archivo .parquet.

    Args:
        datos (list): Lista de diccionarios con la información procesada.
        ruta_parquet (str): Ruta donde se guardará el archivo .parquet.
    """
    df = pd.DataFrame(datos)
    # Selecciona solo las columnas necesarias
    df = df[['nombre_archivo', 'ruta', 'embeddings']]
    df.to_parquet(ruta_parquet, index=False)

def guardar_texto(datos: List[Dict], carpeta_texto: str, formato: str = 'txt'):
    """
    Guarda el texto extraído en archivos de texto plano o JSON.

    Args:
        datos (list): Lista de diccionarios con la información procesada.
        carpeta_texto (str): Carpeta donde se guardarán los archivos de texto.
        formato (str): Formato de los archivos de salida ('txt' o 'json').
    """
    os.makedirs(carpeta_texto, exist_ok=True)
    for dato in datos:
        nombre_archivo = dato['nombre_archivo']
        texto = dato['texto']
        nombre_sin_ext = os.path.splitext(nombre_archivo)[0]
        if formato == 'txt':
            ruta_salida = os.path.join(carpeta_texto, f"{nombre_sin_ext}.txt")
            with open(ruta_salida, 'w', encoding='utf-8') as f:
                f.write(texto)
        elif formato == 'json':
            ruta_salida = os.path.join(carpeta_texto, f"{nombre_sin_ext}.json")
            with open(ruta_salida, 'w', encoding='utf-8') as f:
                json.dump({"texto": texto}, f, ensure_ascii=False, indent=4)
        else:
            print(f"Formato {formato} no soportado. Skipping.")

def main(carpeta_pdfs: str, carpeta_salida_texto: str, ruta_parquet: str, formato_texto: str = 'txt'):
    """
    Función principal para procesar todos los PDFs en una carpeta.

    Args:
        carpeta_pdfs (str): Ruta a la carpeta que contiene los archivos PDF.
        carpeta_salida_texto (str): Ruta a la carpeta donde se guardarán los archivos de texto.
        ruta_parquet (str): Ruta donde se guardará el archivo .parquet.
        formato_texto (str): Formato de los archivos de salida de texto ('txt' o 'json').
    """
    # Lista para almacenar los datos procesados
    datos_procesados = []

    # Obtener la lista de archivos PDF
    pdfs = list(Path(carpeta_pdfs).glob('*.pdf'))
    for ruta_pdf in tqdm(pdfs, desc="Procesando PDFs"):
        dato = procesar_pdf(str(ruta_pdf), nlp, spell, tokenizer, modelo, device)
        datos_procesados.append(dato)
        # Guardar texto inmediatamente para ahorrar memoria
        guardar_texto([dato], carpeta_salida_texto, formato=formato_texto)
    
    # Guardar embeddings en .parquet
    guardar_parquet(datos_procesados, ruta_parquet)
    print(f"Datos guardados en {ruta_parquet}")
    print(f"Textos guardados en {carpeta_salida_texto} en formato {formato_texto}")


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Usando dispositivo: cpu


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
main(
    carpeta_pdfs=pdfs_dir,
    carpeta_salida_texto=output_text_dir,
    ruta_parquet=output_parquet_path,
    formato_texto='json'
)

Procesando PDFs: 100%|██████████| 33/33 [2:12:53<00:00, 241.63s/it]  


Datos guardados en datos_procesados.parquet
Textos guardados en textos en formato json


In [2]:
import polars as pl

df = pl.read_parquet('datos_procesados.parquet')
print(df)

shape: (33, 3)
┌───────────────────────────────────┬───────────────────────────────────┬──────────────────────────┐
│ nombre_archivo                    ┆ ruta                              ┆ embeddings               │
│ ---                               ┆ ---                               ┆ ---                      │
│ str                               ┆ str                               ┆ list[f64]                │
╞═══════════════════════════════════╪═══════════════════════════════════╪══════════════════════════╡
│ DETERMINACION DE LA VARIACION …   ┆ documents\DETERMINACION DE LA …   ┆ [-0.084382, -0.119554, … │
│                                   ┆                                   ┆ -0.17…                   │
│ DISEÑO DE MODULO DIDACTICO DE …   ┆ documents\DISEÑO DE MODULO DID…   ┆ [-0.211153, -0.224628, … │
│                                   ┆                                   ┆ -0.15…                   │
│ PROYECTO GRADO SERGIO Y ANGEL …   ┆ documents\PROYECTO GRADO SERGI…   ┆ [-

In [3]:
import json

# Cargar el archivo JSON
with open('textos/DETERMINACION DE LA VARIACION DEL RENDIMIENTO ENERGETICO DE UN AIRE ACONDICIONADO TIPO MINI SPLIT.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Obtener el texto del JSON
texto = data.get('texto', '')

# Calcular la longitud del texto
longitud_texto = len(texto)
print(f"La longitud del texto es: {longitud_texto}")

La longitud del texto es: 52095


In [4]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

hf_token = os.getenv('HF_TOKEN')

# Reemplaza 'your_huggingface_token' con tu token de acceso personal de HuggingFace
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Verificar si hay una GPU disponible y seleccionar el dispositivo adecuado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo y el tokenizador
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct").to(device)

def extraer_informacion(texto: str, tokenizer, model):
    """
    Extrae entidades reconocidas, resumen corto, palabras clave y tono de escritura de un texto dado.

    Args:
        texto (str): Texto del cual extraer la información.
        tokenizer: Tokenizador del modelo.
        model: Modelo de lenguaje.

    Returns:
        dict: Diccionario con las entidades reconocidas, resumen corto, palabras clave y tono de escritura.
    """
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=20000)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Aquí se asume que el modelo devuelve la información en un formato estructurado
    # como JSON o texto delimitado. Ajusta según sea necesario.
    info = json.loads(decoded_output)
    
    return {
        "entidades_reconocidas": info.get("entidades_reconocidas", []),
        "resumen_corto": info.get("resumen_corto", ""),
        "palabras_clave": info.get("palabras_clave", []),
        "tono_de_escritura": info.get("tono_de_escritura", "")
    }

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path

# Definir la función para procesar un archivo JSON
def procesar_archivo_json(ruta_json: str, tokenizer, model):
    with open(ruta_json, 'r', encoding='utf-8') as f:
        data = json.load(f)
    texto = data.get('texto', '')
    informacion_extraida = extraer_informacion(texto, tokenizer, model)
    return informacion_extraida

# Obtener la lista de archivos JSON en la carpeta 'textos'
carpeta_textos = Path(output_text_dir)
archivos_json = list(carpeta_textos.glob('*.json'))

# Lista para almacenar los resultados
resultados = []

# Procesar cada archivo JSON
for archivo_json in archivos_json:
    informacion = procesar_archivo_json(str(archivo_json), llama_tokenizer, llama_model)
    informacion['nombre_archivo'] = archivo_json.name
    resultados.append(informacion)

# Crear un dataframe de Polars con los resultados
df_resultados = pl.DataFrame(resultados)
print(df_resultados)

In [12]:
df = pl.read_parquet('datos_procesados.parquet')
print(len(df[0,2]))

768


In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, MilvusClient

client = MilvusClient("milvus_demo.db")
client.create_collection(
    collection_name = "document_collection",
    dimension = 768
)
